In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import json

import openmeteo_requests
from collections import OrderedDict
import requests_cache
import pandas as pd
from retry_requests import retry

import pytz
from datetime import timedelta

from entsoe import EntsoePandasClient
from tennet import TenneTClient, DataType, OutputType

with open(".credentials.json") as f:
    credentials = json.load(f)

# Weather Data

In [2]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)


In [3]:
locations = OrderedDict(
    [
        ("Maastricht", [50.851368, 5.690973]),
        ("Groningen", [53.21917, 6.56667]),
        ("Den Helder", [52.9562808, 4.7607972]),
        ("De Bilt", [52.1445592, 5.17377733378846]),
        ("Middelburg", [51.4987962, 3.610998]),
    ]
)

In [4]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
params = {
	"latitude": [loc[0] for loc in locations.values()],
	"longitude": [loc[1] for loc in locations.values()],
	"start_date": "2022-01-01",
	"end_date": "2024-08-01",
	"minutely_15": ["temperature_2m", "wind_speed_10m", "shortwave_radiation","direct_normal_irradiance","diffuse_radiation"],
	"timezone": "auto"
}

In [5]:
responses = openmeteo.weather_api(url, params=params)

In [6]:
# Process first location. Add a for-loop for multiple locations or weather models
loc_data = {}
for response, location in zip(responses, locations.keys()):
    
    loc_data[location] = {"latitude": response.Latitude(), "longitude": response.Longitude()}


    # Process hourly data. The order of variables needs to be the same as requested.
    minutely_15 = response.Minutely15()
    minutely_15_temperature_2m = minutely_15.Variables(0).ValuesAsNumpy()
    minutely_15_wind_speed_10m = minutely_15.Variables(1).ValuesAsNumpy()
    minutely_15_shortwave_radiation = minutely_15.Variables(2).ValuesAsNumpy()
    minutely_15_direct_radiation = minutely_15.Variables(3).ValuesAsNumpy()
    minutely_15_diffuse_radiation = minutely_15.Variables(4).ValuesAsNumpy()

    minutely_15_data = {"date": pd.date_range(
        start = pd.to_datetime(minutely_15.Time(), unit = "s", utc = True),
        end = pd.to_datetime(minutely_15.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = minutely_15.Interval()),
        inclusive = "left"
    )}
    minutely_15_data["temperature_2m"] = minutely_15_temperature_2m
    minutely_15_data["wind_speed_10m"] = minutely_15_wind_speed_10m
    minutely_15_data["shortwave_radiation"] = minutely_15_shortwave_radiation
    minutely_15_data["direct_radiation"] = minutely_15_direct_radiation
    minutely_15_data["diffuse_radiation"] = minutely_15_diffuse_radiation

    minutely_15_dataframe = pd.DataFrame(data = minutely_15_data)

    loc_data[location]["weather_data"] = minutely_15_dataframe

In [7]:
df_merged = pd.concat([loc_data[location]["weather_data"] for location in loc_data])
df_merged = df_merged.groupby("date").mean()
df_merged = df_merged.tz_convert("Europe/Amsterdam")
df_merged

,temperature_2m,wind_speed_10m,shortwave_radiation,direct_radiation,diffuse_radiation
date,,,,,
2021-12-31 23:00:00+01:00,11.340000,22.970766,0.0,0.0,0.0
2021-12-31 23:15:00+01:00,11.330000,22.440861,0.0,0.0,0.0
2021-12-31 23:30:00+01:00,11.300000,21.761368,0.0,0.0,0.0
2021-12-31 23:45:00+01:00,11.280001,21.289515,0.0,0.0,0.0
2022-01-01 00:00:00+01:00,11.250000,20.844385,0.0,0.0,0.0
...,...,...,...,...,...
2024-08-01 22:45:00+02:00,18.530001,9.792000,0.0,0.0,0.0
2024-08-01 23:00:00+02:00,18.430000,9.720000,0.0,0.0,0.0
2024-08-01 23:15:00+02:00,18.289999,9.575999,0.0,0.0,0.0


# Price Data

In [8]:
import xml.etree.ElementTree as ET
import pandas as pd
import datetime

day_ahead_price_data = pd.DataFrame()
for year in list(Path().rglob("development/data/*.xml")):
    with open(year) as f:
        xml_data = f.read()
    # Parse the XML data
    root = ET.fromstring(xml_data)

    # Define the namespace
    ns = {'ns': 'urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:0'}

    # Find the TimeSeries elements
    time_series = root.findall('.//ns:TimeSeries', namespaces=ns)

    # Initialize a list to store the data
    data = []

    # Loop through TimeSeries elements to find Period -> Points -> price.amount
    for series in time_series:
        periods = series.findall('.//ns:Period', namespaces=ns)
        
        for period in periods:
            # Get the start time from the Period's timeInterval
            start_time_str = period.find('ns:timeInterval/ns:start', namespaces=ns).text
            start_time = pd.to_datetime(start_time_str)

            points = period.findall('.//ns:Point', namespaces=ns)
            
            for point in points:
                position = int(point.find('ns:position', namespaces=ns).text)
                price_amount = float(point.find('ns:price.amount', namespaces=ns).text)

                # Calculate the timestamp for each point (position - 1, as position starts at 1)
                timestamp = start_time + pd.Timedelta(hours=position - 1)
                
                data.append({'position': position, 'price.amount': price_amount, 'timestamp': timestamp})

    # Convert the data into a pandas DataFrame
    df = pd.DataFrame(data)

    day_ahead_price_data = pd.concat([day_ahead_price_data,df])
    
day_ahead_price_data = day_ahead_price_data.set_index("timestamp").sort_index().drop(columns="position").rename(columns={"price.amount": "day ahead price"})
day_ahead_price_data = day_ahead_price_data.tz_convert("Europe/Amsterdam")

In [9]:
# day_ahead_price_data.to_feather(f"day_ahead_prices-{day_ahead_price_data.index[0].strftime('%Y-%m-%d')}-{day_ahead_price_data.index[-1].strftime('%Y-%m-%d')}.feather")

In [10]:
start_date = pd.Timestamp(df_merged.index.min().date())
# start_date = pd.Timestamp("2024-10-01")
end_date = pd.Timestamp(df_merged.index.max().date())
# end_date = pd.Timestamp("2024-10-05")

In [11]:

# initiate the client, you can specify a default output to not always specify it per call
client = TenneTClient(default_output=OutputType.CSV)
# retrieve data as text in default output (in this case csv)
# tennet_imbalance_data = client.query_df(DataType.settlementprices, d_from=start_date, d_to=end_date)
tennet_minute_imbalance_data = client.query_df(DataType.balansdeltaprices, d_from=start_date, d_to=end_date)
tennet_ladder_size = client.query_df(DataType.laddersizetotal, d_from=start_date, d_to=end_date)

In [12]:
from tqdm.auto import tqdm
tennet_available_reserve_capacity = pd.DataFrame()
for date in tqdm(pd.date_range(start=start_date, end=end_date, freq="D").strftime("%Y%m%d")):
    try:
        tennet_available_reserve_capacity = pd.concat([tennet_available_reserve_capacity, pd.read_xml(f"https://www.tennet.org/xml/brov/{date}.xml")])
    except:
        continue    

  0%|          | 0/945 [00:00<?, ?it/s]

In [13]:
tennet_available_reserve_capacity

,DATE,PTU,OpregelenMWBeschikbaar,OpregelenMWAfgeroepen,OpregelenMWNietMeerAfroepbaar,AfregelenMWBeschikbaar,AfregelenMWAfgeroepen,AfregelenMWNietMeerAfroepbaar
0,2021-12-31T00:00:00,1,0.0,0,1514.0,0,0,600
1,2021-12-31T00:00:00,2,0.0,0,1564.0,0,0,600
2,2021-12-31T00:00:00,3,0.0,0,1574.0,0,0,600
3,2021-12-31T00:00:00,4,0.0,0,1674.0,0,0,600
4,2021-12-31T00:00:00,5,0.0,0,1724.0,0,0,550
...,...,...,...,...,...,...,...,...
91,2024-08-01T00:00:00,92,0.0,0,1341.0,0,0,3205
92,2024-08-01T00:00:00,93,0.0,0,1461.0,0,0,2880
93,2024-08-01T00:00:00,94,0.0,0,1461.0,0,0,2830
94,2024-08-01T00:00:00,95,0.0,0,1461.0,0,0,2830


In [14]:
from pytz.exceptions import NonExistentTimeError, AmbiguousTimeError
tz = pytz.timezone("Europe/Amsterdam")
# Function to convert DATE and PTU to localized timestamp
def calculate_localized_time(row):
    base_date = row['DATE']
    ptu = row['PTU']
    
    # Each PTU represents a 15-minute interval
    # PTU 1 corresponds to the first 15 minutes of the day
    # Subtract 1 from PTU to align the time offset correctly
    minutes_offset = (ptu - 1) * 15
    
    # Calculate the base time without timezone information
    naive_time = base_date + timedelta(minutes=minutes_offset)
    
    try:
        # Try to localize the time, accounting for DST transitions
        localized_time = tz.localize(naive_time, is_dst=None)
    except NonExistentTimeError:
        # If the time doesn't exist (due to DST forward jump), shift it forward 1 hour
        localized_time = tz.localize(naive_time + timedelta(hours=1), is_dst=True)
    except AmbiguousTimeError:
        # Handle ambiguous times (typically during the fall when the clock repeats one hour)
        localized_time = tz.localize(naive_time, is_dst=True)
    
    return localized_time

In [15]:
tennet_available_reserve_capacity_transformed = tennet_available_reserve_capacity.copy()
tennet_available_reserve_capacity_transformed["DATE"] = pd.to_datetime(tennet_available_reserve_capacity_transformed["DATE"])

tennet_available_reserve_capacity_transformed['timestamp'] = tennet_available_reserve_capacity_transformed.apply(calculate_localized_time, axis=1)
tennet_available_reserve_capacity_transformed= tennet_available_reserve_capacity_transformed.drop(columns=["DATE","PTU"])
tennet_available_reserve_capacity_transformed = tennet_available_reserve_capacity_transformed.drop_duplicates("timestamp")
tennet_available_reserve_capacity_transformed = tennet_available_reserve_capacity_transformed.set_index("timestamp")
tennet_available_reserve_capacity_transformed

,OpregelenMWBeschikbaar,OpregelenMWAfgeroepen,OpregelenMWNietMeerAfroepbaar,AfregelenMWBeschikbaar,AfregelenMWAfgeroepen,AfregelenMWNietMeerAfroepbaar
timestamp,,,,,,
2021-12-31 00:00:00+01:00,0.0,0,1514.0,0,0,600
2021-12-31 00:15:00+01:00,0.0,0,1564.0,0,0,600
2021-12-31 00:30:00+01:00,0.0,0,1574.0,0,0,600
2021-12-31 00:45:00+01:00,0.0,0,1674.0,0,0,600
2021-12-31 01:00:00+01:00,0.0,0,1724.0,0,0,550
...,...,...,...,...,...,...
2024-08-01 22:45:00+02:00,0.0,0,1341.0,0,0,3205
2024-08-01 23:00:00+02:00,0.0,0,1461.0,0,0,2880
2024-08-01 23:15:00+02:00,0.0,0,1461.0,0,0,2830


In [16]:
tennet_ladder_size_transformed = tennet_ladder_size.copy()
tennet_ladder_size_transformed["Date"] = pd.to_datetime(tennet_ladder_size_transformed["Date"])
tennet_ladder_size_transformed=tennet_ladder_size_transformed.rename(columns={"PTE": "PTU", "Date": "DATE"})
tennet_ladder_size_transformed['timestamp'] = tennet_ladder_size_transformed.apply(calculate_localized_time, axis=1)

tennet_ladder_size_transformed= tennet_ladder_size_transformed.drop(columns=["DATE","PTU"])
tennet_ladder_size_transformed = tennet_ladder_size_transformed.drop_duplicates("timestamp")
tennet_ladder_size_transformed = tennet_ladder_size_transformed.set_index("timestamp")
tennet_ladder_size_transformed

,period_from,period_until,af_reg_60,af_reg_15_60,af_reg_0_15,op_reg_0_15,op_reg_15_60,op_reg_60_240,op_reg_240_480,op_reg_480
timestamp,,,,,,,,,,
2021-12-31 00:00:00+01:00,00:00,00:15,0.0,0.0,-785,302,0.0,0.0,0.0,0.0
2021-12-31 00:15:00+01:00,00:15,00:30,0.0,0.0,-783,302,0.0,0.0,0.0,0.0
2021-12-31 00:30:00+01:00,00:30,00:45,0.0,0.0,-731,526,0.0,0.0,0.0,0.0
2021-12-31 00:45:00+01:00,00:45,01:00,0.0,0.0,-674,570,0.0,0.0,0.0,0.0
2021-12-31 01:00:00+01:00,01:00,01:15,0.0,0.0,-684,302,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2024-08-01 22:45:00+02:00,22:45,23:00,0.0,0.0,-819,384,0.0,0.0,0.0,0.0
2024-08-01 23:00:00+02:00,23:00,23:15,0.0,0.0,-822,382,0.0,0.0,0.0,0.0
2024-08-01 23:15:00+02:00,23:15,23:30,0.0,0.0,-822,383,0.0,0.0,0.0,0.0


In [17]:
tennet_minute_imbalance_data = tennet_minute_imbalance_data.set_index("timestamp").tz_localize("Europe/Amsterdam", ambiguous="infer")
tennet_minute_imbalance_data

,To regulate up,To regulate down,To regulate up_reserve,To regulate down_reserve,Emergency capacity,Highest_price_upward,Lowest_price_downward
timestamp,,,,,,,
2021-12-31 00:00:00+01:00,0.0,128.0,0.0,0.0,0,0.0,-63.51
2021-12-31 00:01:00+01:00,0.0,135.0,0.0,0.0,0,0.0,-78.08
2021-12-31 00:02:00+01:00,0.0,136.0,0.0,0.0,0,0.0,-78.08
2021-12-31 00:03:00+01:00,0.0,110.0,0.0,0.0,0,0.0,-63.51
2021-12-31 00:04:00+01:00,0.0,84.0,0.0,0.0,0,0.0,-3.75
...,...,...,...,...,...,...,...
2024-08-01 23:55:00+02:00,0.0,44.0,0.0,0.0,0,0.0,77.06
2024-08-01 23:56:00+02:00,0.0,64.0,0.0,0.0,0,0.0,76.25
2024-08-01 23:57:00+02:00,0.0,88.0,0.0,0.0,0,0.0,72.69


In [18]:
# start_date = pd.Timestamp("2024-09-01")
# end_date = pd.Timestamp("2024-10-01")
client = EntsoePandasClient(api_key=credentials["entso-e-key"], retry_count=5, retry_delay=5)

# imbalance_data = client.query_imbalance_prices("NL", start=start_date.tz_localize("Europe/Amsterdam"), end=end_date.tz_localize("Europe/Amsterdam"), psr_type=None)
# reserve_price_capacity=client.query_contracted_reserve_prices_procured_capacity(country_code="NL", start=start_date.tz_localize("Europe/Amsterdam"),end=end_date.tz_localize("Europe/Amsterdam"),type_marketagreement_type="A01",process_type="A51", psr_type=None)
# balancing_capacity=client.query_procured_balancing_capacity(country_code="NL", start=start_date.tz_localize("Europe/Amsterdam"), end=end_date.tz_localize("Europe/Amsterdam"),process_type="A51", type_marketagreement_type=None)

# day_ahead_data = client.query_day_ahead_prices("NL", start=start_date.tz_localize("Europe/Amsterdam"), end=end_date.tz_localize("Europe/Amsterdam"))
load_data = client.query_load_forecast("NL", start=start_date.tz_localize("Europe/Amsterdam"), end=end_date.tz_localize("Europe/Amsterdam"))
wind_solar_data = client.query_intraday_wind_and_solar_forecast("NL", start=start_date.tz_localize("Europe/Amsterdam"), end=end_date.tz_localize("Europe/Amsterdam"), psr_type=None)
production_data = client.query_generation_forecast("NL", start=start_date.tz_localize("Europe/Amsterdam"), end=end_date.tz_localize("Europe/Amsterdam"))

reserve_price=client.query_contracted_reserve_prices(country_code="NL", start=start_date.tz_localize("Europe/Amsterdam"), end=end_date.tz_localize("Europe/Amsterdam"),type_marketagreement_type="A01", psr_type=None)
reserve_amount=client.query_contracted_reserve_amount(country_code="NL", start=start_date.tz_localize("Europe/Amsterdam"), end=end_date.tz_localize("Europe/Amsterdam"),type_marketagreement_type="A01", psr_type=None)

/home/gergo/miniforge3/envs/simpl-lgbm-cloud/lib/python3.10/site-packages/entsoe/parsers.py:449: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level = [0,1]).mean()
/home/gergo/miniforge3/envs/simpl-lgbm-cloud/lib/python3.10/site-packages/entsoe/parsers.py:449: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level = [0,1]).mean()
/home/gergo/miniforge3/envs/simpl-lgbm-cloud/lib/python3.10/site-packages/entsoe/parsers.py:449: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level = [0,1]).mean()
/home/gergo/miniforge3/envs/simpl-lgbm-cloud/lib/python3.10/site-packages/entsoe/parsers.py:449: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level = [0

In [19]:
reserve_price.columns = reserve_price.columns.map(' - '.join).str.strip(' - ')
reserve_amount.columns = reserve_amount.columns.map(' - '.join).str.strip(' - ')

# Merge

In [33]:
all_data = (
    tennet_minute_imbalance_data.join(df_merged, how="outer")
    .interpolate("akima", limit_direction="forward")
)
all_data = all_data.drop(columns=["To regulate up", "To regulate down", "To regulate up_reserve", "To regulate down_reserve", "Emergency capacity"])
# all_data = all_data.join(day_ahead_data.rename("day_ahead_price"), how="outer")
all_data = all_data.join(day_ahead_price_data, how="left")
# all_data["day_ahead_price"] = saved_data["day_ahead_price"]
all_data = all_data.join(load_data, how="left")
all_data = all_data.join(wind_solar_data, how="left")
all_data = all_data.join(production_data.rename("production_forecast"), how="left")
# all_data["non_tz_time"] = all_data.index.tz_localize(None)
# all_data = all_data.merge(tennet_available_reserve_capacity,left_on="non_tz_time",right_index=True, how="left")
# all_data = all_data.merge(tennet_ladder_size,left_on="non_tz_time",right_index=True, how="left")
# all_data = all_data.join(reserve_price, how="left")
# all_data = all_data.join(reserve_amount, lsuffix=" price", rsuffix=" amount", how="left")
all_data = all_data.join(tennet_available_reserve_capacity_transformed,how="left")
all_data = all_data.join(tennet_ladder_size_transformed, how="left")
all_data = all_data.join(reserve_price, how="left")
all_data = all_data.join(reserve_amount, lsuffix=" price", rsuffix=" amount", how="left")
all_data = all_data.rename(columns={"Forecasted Load": "load_forecast", "Solar": "solar_forecast", "Wind Offshore": "wind_offshore", "Wind Onshore": "wind_onshore"})
# all_data = all_data.drop(columns=["non_tz_time"])
all_data = all_data.drop(columns=["period_from", "period_until"])
all_data = all_data.ffill(limit_area="inside")
# all_data["day_ahead_price"] = all_data["day_ahead_price"].ffill(limit_area="inside")
# all_data["load_forecast"] = all_data["load_forecast"].ffill(limit_area="inside")
# all_data["solar_forecast"] = all_data["solar_forecast"].ffill(limit_area="inside")
# all_data["wind_offshore"] = all_data["wind_offshore"].ffill(limit_area="inside")
# all_data["wind_onshore"] = all_data["wind_onshore"].ffill(limit_area="inside")
# all_data["production_forecast"] = all_data["production_forecast"].ffill(limit_area="inside")


all_data = all_data.dropna()
all_data

,Highest_price_upward,Lowest_price_downward,temperature_2m,wind_speed_10m,shortwave_radiation,direct_radiation,diffuse_radiation,day ahead price,load_forecast,solar_forecast,...,Automatic frequency restoration reserve - Down price,Automatic frequency restoration reserve - Up price,Frequency containment reserve - Symmetric price,Manual frequency restoration reserve - Down price,Manual frequency restoration reserve - Up price,Automatic frequency restoration reserve - Down amount,Automatic frequency restoration reserve - Up amount,Frequency containment reserve - Symmetric amount,Manual frequency restoration reserve - Down amount,Manual frequency restoration reserve - Up amount
2021-12-31 23:00:00+01:00,166.27,0.00,11.340000,22.970766,0.0,0.0,0.0,98.20,12352.0,0.0,...,17.89,14.55,13.68,9.24,3.79,370.0,290.0,76.0,680.0,1015.0
2021-12-31 23:01:00+01:00,168.30,0.00,11.339970,22.940042,0.0,0.0,0.0,98.20,12352.0,0.0,...,17.89,14.55,13.68,9.24,3.79,370.0,290.0,76.0,680.0,1015.0
2021-12-31 23:02:00+01:00,171.89,0.00,11.339873,22.908567,0.0,0.0,0.0,98.20,12352.0,0.0,...,17.89,14.55,13.68,9.24,3.79,370.0,290.0,76.0,680.0,1015.0
2021-12-31 23:03:00+01:00,173.94,0.00,11.339706,22.876364,0.0,0.0,0.0,98.20,12352.0,0.0,...,17.89,14.55,13.68,9.24,3.79,370.0,290.0,76.0,680.0,1015.0
2021-12-31 23:04:00+01:00,177.85,0.00,11.339462,22.843451,0.0,0.0,0.0,98.20,12352.0,0.0,...,17.89,14.55,13.68,9.24,3.79,370.0,290.0,76.0,680.0,1015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-30 23:56:00+02:00,0.00,57.68,20.906435,9.642559,0.0,0.0,0.0,99.04,11098.0,0.0,...,5.95,10.18,4.57,1.39,1.22,406.0,373.0,38.0,670.0,931.0
2024-07-30 23:57:00+02:00,0.00,45.90,20.897013,9.627839,0.0,0.0,0.0,99.04,11098.0,0.0,...,5.95,10.18,4.57,1.39,1.22,406.0,373.0,38.0,670.0,931.0
2024-07-30 23:58:00+02:00,0.00,38.61,20.887791,9.611839,0.0,0.0,0.0,99.04,11098.0,0.0,...,5.95,10.18,4.57,1.39,1.22,406.0,373.0,38.0,670.0,931.0
2024-07-30 23:59:00+02:00,0.00,9.87,20.878782,9.594559,0.0,0.0,0.0,99.04,11098.0,0.0,...,5.95,10.18,4.57,1.39,1.22,406.0,373.0,38.0,670.0,931.0


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly_resampler import register_plotly_resampler

plotted_df = all_data.loc["2024-06-01":]
# Create a figure with subplots
fig = make_subplots(rows=len(plotted_df.columns), cols=1, shared_xaxes=True, vertical_spacing=0.005)


# Plot each y column in a separate subplot
for i, column in enumerate(plotted_df.columns):
    fig.add_trace(go.Scatter(x=plotted_df.index, y=plotted_df[column], name=column), row=i+1, col=1)

# Update layout
fig.update_layout(height=1000, width=1600, title_text="All Data")
fig.update_xaxes(title_text="Times")
fig.update_yaxes(title_text="Values", row=4, col=1)  # Update the y-axis title of the middle subplot

# Show the figure
fig.show()

In [34]:
all_data = all_data.reset_index(names="times")

In [35]:
all_data

,times,Highest_price_upward,Lowest_price_downward,temperature_2m,wind_speed_10m,shortwave_radiation,direct_radiation,diffuse_radiation,day ahead price,load_forecast,...,Automatic frequency restoration reserve - Down price,Automatic frequency restoration reserve - Up price,Frequency containment reserve - Symmetric price,Manual frequency restoration reserve - Down price,Manual frequency restoration reserve - Up price,Automatic frequency restoration reserve - Down amount,Automatic frequency restoration reserve - Up amount,Frequency containment reserve - Symmetric amount,Manual frequency restoration reserve - Down amount,Manual frequency restoration reserve - Up amount
0,2021-12-31 23:00:00+01:00,166.27,0.00,11.340000,22.970766,0.0,0.0,0.0,98.20,12352.0,...,17.89,14.55,13.68,9.24,3.79,370.0,290.0,76.0,680.0,1015.0
1,2021-12-31 23:01:00+01:00,168.30,0.00,11.339970,22.940042,0.0,0.0,0.0,98.20,12352.0,...,17.89,14.55,13.68,9.24,3.79,370.0,290.0,76.0,680.0,1015.0
2,2021-12-31 23:02:00+01:00,171.89,0.00,11.339873,22.908567,0.0,0.0,0.0,98.20,12352.0,...,17.89,14.55,13.68,9.24,3.79,370.0,290.0,76.0,680.0,1015.0
3,2021-12-31 23:03:00+01:00,173.94,0.00,11.339706,22.876364,0.0,0.0,0.0,98.20,12352.0,...,17.89,14.55,13.68,9.24,3.79,370.0,290.0,76.0,680.0,1015.0
4,2021-12-31 23:04:00+01:00,177.85,0.00,11.339462,22.843451,0.0,0.0,0.0,98.20,12352.0,...,17.89,14.55,13.68,9.24,3.79,370.0,290.0,76.0,680.0,1015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1356476,2024-07-30 23:56:00+02:00,0.00,57.68,20.906435,9.642559,0.0,0.0,0.0,99.04,11098.0,...,5.95,10.18,4.57,1.39,1.22,406.0,373.0,38.0,670.0,931.0
1356477,2024-07-30 23:57:00+02:00,0.00,45.90,20.897013,9.627839,0.0,0.0,0.0,99.04,11098.0,...,5.95,10.18,4.57,1.39,1.22,406.0,373.0,38.0,670.0,931.0
1356478,2024-07-30 23:58:00+02:00,0.00,38.61,20.887791,9.611839,0.0,0.0,0.0,99.04,11098.0,...,5.95,10.18,4.57,1.39,1.22,406.0,373.0,38.0,670.0,931.0
1356479,2024-07-30 23:59:00+02:00,0.00,9.87,20.878782,9.594559,0.0,0.0,0.0,99.04,11098.0,...,5.95,10.18,4.57,1.39,1.22,406.0,373.0,38.0,670.0,931.0


# Saving

In [36]:
all_data.to_csv(f"development/data/imbalance_data_{all_data['times'].iloc[0].date()}-{all_data['times'].iloc[-1].date()}.csv", index=False)

In [37]:
saved_data = pd.read_csv("development/data/imbalance_data_2021-12-31-2024-07-31.csv")
saved_data

,times,Highest_price_upward,Lowest_price_downward,temperature_2m,wind_speed_10m,shortwave_radiation,direct_radiation,diffuse_radiation,day ahead price,load_forecast,...,Automatic frequency restoration reserve - Down price,Automatic frequency restoration reserve - Up price,Frequency containment reserve - Symmetric price,Manual frequency restoration reserve - Down price,Manual frequency restoration reserve - Up price,Automatic frequency restoration reserve - Down amount,Automatic frequency restoration reserve - Up amount,Frequency containment reserve - Symmetric amount,Manual frequency restoration reserve - Down amount,Manual frequency restoration reserve - Up amount
0,2021-12-31 23:00:00+01:00,166.27,0.00,11.340000,22.970766,0.0,0.0,0.0,98.20,12352.0,...,17.89,14.55,13.68,9.24,3.79,370.0,290.0,76.0,680.0,1015.0
1,2021-12-31 23:01:00+01:00,168.30,0.00,11.339970,22.940042,0.0,0.0,0.0,98.20,12352.0,...,17.89,14.55,13.68,9.24,3.79,370.0,290.0,76.0,680.0,1015.0
2,2021-12-31 23:02:00+01:00,171.89,0.00,11.339873,22.908567,0.0,0.0,0.0,98.20,12352.0,...,17.89,14.55,13.68,9.24,3.79,370.0,290.0,76.0,680.0,1015.0
3,2021-12-31 23:03:00+01:00,173.94,0.00,11.339706,22.876364,0.0,0.0,0.0,98.20,12352.0,...,17.89,14.55,13.68,9.24,3.79,370.0,290.0,76.0,680.0,1015.0
4,2021-12-31 23:04:00+01:00,177.85,0.00,11.339462,22.843450,0.0,0.0,0.0,98.20,12352.0,...,17.89,14.55,13.68,9.24,3.79,370.0,290.0,76.0,680.0,1015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1356476,2024-07-30 23:56:00+02:00,0.00,57.68,20.906435,9.642559,0.0,0.0,0.0,99.04,11098.0,...,5.95,10.18,4.57,1.39,1.22,406.0,373.0,38.0,670.0,931.0
1356477,2024-07-30 23:57:00+02:00,0.00,45.90,20.897013,9.627839,0.0,0.0,0.0,99.04,11098.0,...,5.95,10.18,4.57,1.39,1.22,406.0,373.0,38.0,670.0,931.0
1356478,2024-07-30 23:58:00+02:00,0.00,38.61,20.887790,9.611839,0.0,0.0,0.0,99.04,11098.0,...,5.95,10.18,4.57,1.39,1.22,406.0,373.0,38.0,670.0,931.0
1356479,2024-07-30 23:59:00+02:00,0.00,9.87,20.878782,9.594559,0.0,0.0,0.0,99.04,11098.0,...,5.95,10.18,4.57,1.39,1.22,406.0,373.0,38.0,670.0,931.0
